In [16]:
 # Import modules
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

import sqlite3

In [17]:
conn = sqlite3.connect('data/sqlite/nfl_data.sqlite')
df = pd.read_sql('SELECT * FROM relative_team_formations', conn).set_index('play_uuid').dropna()

# Review the DataFrame
df.head()

,off_x0,off_y0,off_s0,off_a0,off_dis0,off_o0,off_dir0,off_x1,off_y1,off_s1,...,def_o9,def_dir9,def_x10,def_y10,def_s10,def_a10,def_dis10,def_o10,def_dir10,converted
play_uuid,,,,,,,,,,,,,,,,,,,,,
2022090800.101,-1.089999,-1.850000,0.10,1.14,0.02,273.97,348.75,-0.459999,-0.050000,0.0,...,32.14,352.71,11.180001,5.200000,0.06,0.03,0.01,79.04,59.57,0
2022090800.1187,-1.650000,-0.070001,0.05,0.85,0.01,272.46,55.53,-0.660000,13.919999,0.0,...,88.65,101.47,0.780000,-7.460001,0.94,0.81,0.09,87.20,10.33,0
2022090800.1254,-5.120001,-0.249999,0.00,0.00,0.00,272.98,143.12,-0.910001,-14.539999,0.0,...,89.72,83.33,1.229999,21.140001,0.04,0.04,0.02,131.60,216.90,0
2022090800.1385,-1.649998,-0.230000,0.08,0.88,0.01,273.91,130.33,-0.579998,7.250000,0.0,...,86.08,84.70,0.150002,-10.100000,0.00,0.00,0.03,49.70,91.93,1
2022090800.1406,-1.260001,0.010000,0.25,1.67,0.02,278.66,52.44,-0.280001,-13.810000,0.0,...,78.75,82.94,1.269999,7.740000,0.32,0.18,0.03,117.94,100.45,0


In [18]:
 # Seperate the features, X,  from the target variable, y
y = df['converted']
X = df.drop(columns=['converted'])

In [19]:
X

,off_x0,off_y0,off_s0,off_a0,off_dis0,off_o0,off_dir0,off_x1,off_y1,off_s1,...,def_dis9,def_o9,def_dir9,def_x10,def_y10,def_s10,def_a10,def_dis10,def_o10,def_dir10
play_uuid,,,,,,,,,,,,,,,,,,,,,
2022090800.101,-1.089999,-1.850000,0.10,1.14,0.02,273.97,348.75,-0.459999,-0.050000,0.00,...,0.05,32.14,352.71,11.180001,5.200000,0.06,0.03,0.01,79.04,59.57
2022090800.1187,-1.650000,-0.070001,0.05,0.85,0.01,272.46,55.53,-0.660000,13.919999,0.00,...,0.01,88.65,101.47,0.780000,-7.460001,0.94,0.81,0.09,87.20,10.33
2022090800.1254,-5.120001,-0.249999,0.00,0.00,0.00,272.98,143.12,-0.910001,-14.539999,0.00,...,0.01,89.72,83.33,1.229999,21.140001,0.04,0.04,0.02,131.60,216.90
2022090800.1385,-1.649998,-0.230000,0.08,0.88,0.01,273.91,130.33,-0.579998,7.250000,0.00,...,0.01,86.08,84.70,0.150002,-10.100000,0.00,0.00,0.03,49.70,91.93
2022090800.1406,-1.260001,0.010000,0.25,1.67,0.02,278.66,52.44,-0.280001,-13.810000,0.00,...,0.01,78.75,82.94,1.269999,7.740000,0.32,0.18,0.03,117.94,100.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022110700.612,-0.930002,-1.250000,0.05,0.44,0.01,74.74,48.42,-1.260002,-2.790000,0.00,...,0.02,294.68,10.42,7.429998,9.300000,0.22,0.56,0.02,240.21,321.26
2022110700.670,-0.979999,-1.369999,0.13,0.90,0.01,92.18,120.21,-1.169999,-2.789999,0.06,...,0.11,310.54,326.89,1.120001,18.770001,0.00,0.00,0.00,253.94,152.31
2022110700.80,-4.479998,0.030000,0.01,0.02,0.03,277.43,326.30,-0.779998,-3.280000,0.01,...,0.01,66.66,95.41,12.010002,-5.740000,0.45,0.25,0.04,86.05,318.34


In [20]:
X = pd.get_dummies(X)
X.head()

,off_x0,off_y0,off_s0,off_a0,off_dis0,off_o0,off_dir0,off_x1,off_y1,off_s1,...,def_dis9,def_o9,def_dir9,def_x10,def_y10,def_s10,def_a10,def_dis10,def_o10,def_dir10
play_uuid,,,,,,,,,,,,,,,,,,,,,
2022090800.101,-1.089999,-1.850000,0.10,1.14,0.02,273.97,348.75,-0.459999,-0.050000,0.0,...,0.05,32.14,352.71,11.180001,5.200000,0.06,0.03,0.01,79.04,59.57
2022090800.1187,-1.650000,-0.070001,0.05,0.85,0.01,272.46,55.53,-0.660000,13.919999,0.0,...,0.01,88.65,101.47,0.780000,-7.460001,0.94,0.81,0.09,87.20,10.33
2022090800.1254,-5.120001,-0.249999,0.00,0.00,0.00,272.98,143.12,-0.910001,-14.539999,0.0,...,0.01,89.72,83.33,1.229999,21.140001,0.04,0.04,0.02,131.60,216.90
2022090800.1385,-1.649998,-0.230000,0.08,0.88,0.01,273.91,130.33,-0.579998,7.250000,0.0,...,0.01,86.08,84.70,0.150002,-10.100000,0.00,0.00,0.03,49.70,91.93
2022090800.1406,-1.260001,0.010000,0.25,1.67,0.02,278.66,52.44,-0.280001,-13.810000,0.0,...,0.01,78.75,82.94,1.269999,7.740000,0.32,0.18,0.03,117.94,100.45


In [21]:
# Split the dataset using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [22]:
 # Instantiate a StandardScaler instance
scaler = StandardScaler()

# Fit the training data to the standard scaler
X_scaler = scaler.fit(X_train)

# Transform the training data using the scaler
X_train_scaled = X_scaler.transform(X_train)

# Transform the testing data using the scaler
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Import the KNeighborsClassifier module from sklearn
from sklearn.neighbors import KNeighborsClassifier

# Instantiate the KNeighborsClassifier model with n_neighbors = 3
knn = KNeighborsClassifier(n_neighbors=10)

In [24]:
# Train the model using the training data
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=10)

In [25]:
# Create predictions using the testing data
y_pred = knn.predict(X_test_scaled)

In [26]:
print(confusion_matrix(y_test, y_pred))

[[1251    7]
 [ 313    4]]


In [27]:
 # Print the classification report comparing the testing data to the model predictions
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.99      0.89      1258
           1       0.36      0.01      0.02       317

    accuracy                           0.80      1575
   macro avg       0.58      0.50      0.46      1575
weighted avg       0.71      0.80      0.71      1575

